In [1]:
#the first thing is we want to test the loss fucntion
#we have one implementation but likely better to use the jacob schreiber version of all these things

#the first thing we need to do is load in the dataloader and the bias model, we'll ignore the other hyena model right now

#a simple script to test loading the bias model into pytorch
#we have some scripts stored in bpnet lite
import sys
sys.path.append('/data/leslie/sarthak/chrombpnet')
from bpnetlite.bpnet import BPNet

#now let's try loading the model using the model method
model = BPNet.from_chrombpnet('/data/leslie/sarthak/data/chrombpnet_test/bias_model/models/k562_bias.h5')
model
#and this works, just had to comment out those 2 lines lol!
#let's make a random size input
import torch
data = torch.rand(64,4,2114)*10
out = model(data)
print(len(out))
print(out[0].shape) #the profile
print(out[1].shape) #the counts
print(out)

2
torch.Size([64, 1, 1000])
torch.Size([64, 1])
(tensor([[[ 16.2792,  -0.9584,   6.0445,  ...,   6.5974,   6.3012,  -2.5893]],

        [[ 19.6213,  16.6265,   0.9655,  ...,   4.8606,  27.4234,  20.1950]],

        [[  7.2324,  -3.7198,   4.3464,  ...,  18.3869,   9.2766,  18.9896]],

        ...,

        [[  1.2743,  10.4694,  13.3491,  ..., -11.3801,  15.4625,   7.2214]],

        [[ -1.1277,  14.6689,  -0.3529,  ...,   5.8412,  19.1069,  12.7201]],

        [[ 20.2153,   7.3612,   9.4962,  ...,  -5.1227,   9.5780,  -2.8292]]],
       grad_fn=<SliceBackward0>), tensor([[16.4963],
        [16.6755],
        [16.9978],
        [16.5328],
        [16.8903],
        [16.5693],
        [16.4143],
        [16.7434],
        [17.2390],
        [16.5309],
        [16.8418],
        [16.2891],
        [16.7821],
        [16.5837],
        [16.7446],
        [16.7610],
        [16.6290],
        [16.5438],
        [16.6921],
        [17.1096],
        [16.6807],
        [16.5888],
        [16

In [3]:
#now let's see how loss is calculated
def MNLLLoss(logps, true_counts):
	"""A loss function based on the multinomial negative log-likelihood.

	This loss function takes in a tensor of normalized log probabilities such
	that the sum of each row is equal to 1 (e.g. from a log softmax) and
	an equal sized tensor of true counts and returns the probability of
	observing the true counts given the predicted probabilities under a
	multinomial distribution. Can accept tensors with 2 or more dimensions
	and averages over all except for the last axis, which is the number
	of categories.

	Adapted from Alex Tseng.

	Parameters
	----------
	logps: torch.tensor, shape=(n, ..., L)
		A tensor with `n` examples and `L` possible categories. 

	true_counts: torch.tensor, shape=(n, ..., L)
		A tensor with `n` examples and `L` possible categories.

	Returns
	-------
	loss: float
		The multinomial log likelihood loss of the true counts given the
		predicted probabilities, averaged over all examples and all other
		dimensions.
	"""

	log_fact_sum = torch.lgamma(torch.sum(true_counts, dim=-1) + 1)
	log_prod_fact = torch.sum(torch.lgamma(true_counts + 1), dim=-1)
	log_prod_exp = torch.sum(true_counts * logps, dim=-1)
	return -log_fact_sum + log_prod_fact - log_prod_exp

labels_profile = torch.rand(64,1000)*10
print(labels_profile.shape)
print(out[0].shape)

torch.Size([64, 1000])


NameError: name 'out' is not defined

In [6]:
loss = MNLLLoss(out[0].squeeze(), labels_profile)
print(loss.shape)
print(loss)

torch.Size([64])
tensor([-78403.6719, -79383.5625, -76239.7969, -79508.6562, -77803.5547,
        -75716.1406, -80500.1875, -76729.5859, -74137.3516, -79182.4688,
        -76608.4453, -78685.4688, -79915.8672, -81668.2422, -79139.1016,
        -76819.9609, -76397.5156, -78656.1250, -74702.6250, -78717.7969,
        -78379.3594, -75467.5234, -77883.6406, -75138.4062, -74814.8438,
        -81156.3438, -78352.8438, -80276.8984, -78168.0469, -78166.5938,
        -75969.5625, -80774.7266, -77013.3125, -77832.3359, -75345.7188,
        -77301.8750, -74856.0625, -78989.7734, -77662.4297, -74652.0781,
        -74427.6406, -74569.5078, -79467.2578, -77182.9531, -79619.4531,
        -78124.8516, -77646.7422, -74681.9688, -75639.5781, -77759.8828,
        -75653.9062, -78317.3438, -76519.2500, -78632.3750, -77222.9688,
        -76150.3516, -77275.3984, -75491.6484, -76596.8594, -73243.0469,
        -73426.8359, -78307.6719, -74891.5000, -77744.3828],
       grad_fn=<SubBackward0>)


In [7]:
def cbpnet_multinomial_nll(logits,true_counts, len_batch=None, ignore_index=-100, mask = True, weight = .5, pos_weight = .04):
    """
    Compute the multinomial negative log-likelihood in PyTorch.

    Args:
      true_counts: Observed count values (tensor of shape [batch_size, num_classes])
      logits: Predicted logit values (tensor of shape [batch_size, num_classes])
    """
    #check if the input is a tuple
    if isinstance(logits, tuple): #for wandb tracking, it inputs logits here, since then it's a tuple
        logits = logits[0]
    
    # Calculate the total counts per example
    counts_per_example = true_counts.sum(dim=-1)

    # Create the Multinomial distribution with the given logits
    dist = torch.distributions.Multinomial(total_count=counts_per_example, logits=logits)

    # Calculate the negative log probability of the true counts
    log_prob = dist.log_prob(true_counts)

    # Calculate the mean NLL loss over the batch
    loss = -log_prob.mean()

    return loss

loss2 = cbpnet_multinomial_nll(out[0].squeeze(), labels_profile)
print(loss2)

NotImplementedError: inhomogeneous total_count is not supported

In [1]:
#yeah this is not going to work well, let's just say no to this...
#but if we use the schreiber one, do we need to do log first?
#let's just give the same input to both and see what the output is
import sys
sys.path.append('/data/leslie/sarthak/chrombpnet')
from bpnetlite.bpnet import BPNet

model = BPNet.from_chrombpnet('/data/leslie/sarthak/data/chrombpnet_test/bias_model/models/k562_bias.h5')
import torch
import numpy as np
array1 = np.array(range(64*4*2114)).reshape(64,4,2114) / 1000
print(array1.shape)
array2 = np.array(range(64*1000)).reshape(64,1000) / 1000
print(array2.shape)


(64, 4, 2114)
(64, 1000)


In [2]:
#let's now calculate the loss
def cbpnet_multinomial_nll(logits,true_counts, len_batch=None, ignore_index=-100, mask = True, weight = .5, pos_weight = .04):

    """A loss function based on the multinomial negative log-likelihood.
    modified by me to include things like doing thE log softmax

    This loss function takes in a tensor of normalized log probabilities such
    that the sum of each row is equal to 1 (e.g. from a log softmax) and
    an equal sized tensor of true counts and returns the probability of
    observing the true counts given the predicted probabilities under a
    multinomial distribution. Can accept tensors with 2 or more dimensions
    and averages over all except for the last axis, which is the number
    of categories.

    Adapted from Alex Tseng.

    Parameters
    ----------
    logps: torch.tensor, shape=(n, ..., L)
        A tensor with `n` examples and `L` possible categories. 

    true_counts: torch.tensor, shape=(n, ..., L)
        A tensor with `n` examples and `L` possible categories.

    Returns
    -------
    loss: float
        The multinomial log likelihood loss of the true counts given the
        predicted probabilities, averaged over all examples and all other
        dimensions.
    """
    if isinstance(logits, tuple): #for wandb tracking, it inputs logits here, since then it's a tuple
        logits = logits[0]
    logps = torch.log_softmax(logits, dim=-1)
    log_fact_sum = torch.lgamma(torch.sum(true_counts, dim=-1) + 1)
    log_prod_fact = torch.sum(torch.lgamma(true_counts + 1), dim=-1)
    log_prod_exp = torch.sum(true_counts * logps, dim=-1)
    return -log_fact_sum + log_prod_fact - log_prod_exp

fake_logits = np.array(range(64*1000)).reshape(64,1000) / 1000
fake_logits = torch.tensor(fake_logits)
fake_labels = np.array(range(64*1000)).reshape(64,1000) / 1000
fake_labels = torch.tensor(fake_labels)

loss = cbpnet_multinomial_nll(fake_logits, fake_labels)
print(loss)

tensor([ 698.4480, 1170.9492, 1439.0846, 1635.6626, 1795.2100, 1932.0447,
        2053.5070, 2163.8679, 2265.8329, 2361.2251, 2451.3312, 2537.0936,
        2619.2231, 2698.2691, 2774.6652, 2848.7593, 2920.8352, 2991.1271,
        3059.8307, 3127.1112, 3193.1092, 3257.9456, 3321.7249, 3384.5379,
        3446.4645, 3507.5749, 3567.9314, 3627.5892, 3686.5979, 3745.0019,
        3802.8411, 3860.1517, 3916.9666, 3973.3155, 4029.2257, 4084.7220,
        4139.8273, 4194.5626, 4248.9472, 4302.9991, 4356.7347, 4410.1694,
        4463.3175, 4516.1922, 4568.8058, 4621.1699, 4673.2953, 4725.1921,
        4776.8697, 4828.3372, 4879.6027, 4930.6743, 4981.5594, 5032.2648,
        5082.7974, 5133.1633, 5183.3684, 5233.4184, 5283.3187, 5333.0741,
        5382.6897, 5432.1699, 5481.5190, 5530.7413], dtype=torch.float64)


In [3]:
print(loss.mean()) #so we do indeed need to average it!

tensor(3744.9462, dtype=torch.float64)


In [4]:
#what if we don'to do the log softmax?

#let's now calculate the loss
def cbpnet_multinomial_nll(logits,true_counts, len_batch=None, ignore_index=-100, mask = True, weight = .5, pos_weight = .04):

    """A loss function based on the multinomial negative log-likelihood.
    modified by me to include things like doing thE log softmax

    This loss function takes in a tensor of normalized log probabilities such
    that the sum of each row is equal to 1 (e.g. from a log softmax) and
    an equal sized tensor of true counts and returns the probability of
    observing the true counts given the predicted probabilities under a
    multinomial distribution. Can accept tensors with 2 or more dimensions
    and averages over all except for the last axis, which is the number
    of categories.

    Adapted from Alex Tseng.

    Parameters
    ----------
    logps: torch.tensor, shape=(n, ..., L)
        A tensor with `n` examples and `L` possible categories. 

    true_counts: torch.tensor, shape=(n, ..., L)
        A tensor with `n` examples and `L` possible categories.

    Returns
    -------
    loss: float
        The multinomial log likelihood loss of the true counts given the
        predicted probabilities, averaged over all examples and all other
        dimensions.
    """
    if isinstance(logits, tuple): #for wandb tracking, it inputs logits here, since then it's a tuple
        logits = logits[0]
    logps = logits
    log_fact_sum = torch.lgamma(torch.sum(true_counts, dim=-1) + 1)
    log_prod_fact = torch.sum(torch.lgamma(true_counts + 1), dim=-1)
    log_prod_exp = torch.sum(true_counts * logps, dim=-1)
    return -log_fact_sum + log_prod_fact - log_prod_exp

fake_logits = np.array(range(64*1000)).reshape(64,1000) / 1000
fake_logits = torch.tensor(fake_logits)
fake_labels = np.array(range(64*1000)).reshape(64,1000) / 1000
fake_labels = torch.tensor(fake_labels)

loss = cbpnet_multinomial_nll(fake_logits, fake_labels)
print(loss)
print(loss.mean())
#we get a massive number now... it claims we need the log softmax but I don't see it in the chrombpnet model

tensor([-3.0221e+03, -1.1498e+04, -2.2178e+04, -3.4929e+04, -4.9718e+04,
        -6.6529e+04, -8.5356e+04, -1.0619e+05, -1.2904e+05, -1.5389e+05,
        -1.8075e+05, -2.0961e+05, -2.4048e+05, -2.7335e+05, -3.0822e+05,
        -3.4509e+05, -3.8397e+05, -4.2485e+05, -4.6773e+05, -5.1261e+05,
        -5.5949e+05, -6.0837e+05, -6.5926e+05, -7.1214e+05, -7.6703e+05,
        -8.2391e+05, -8.8280e+05, -9.4369e+05, -1.0066e+06, -1.0715e+06,
        -1.1384e+06, -1.2073e+06, -1.2781e+06, -1.3510e+06, -1.4259e+06,
        -1.5028e+06, -1.5817e+06, -1.6626e+06, -1.7455e+06, -1.8304e+06,
        -1.9173e+06, -2.0062e+06, -2.0971e+06, -2.1900e+06, -2.2849e+06,
        -2.3818e+06, -2.4807e+06, -2.5816e+06, -2.6845e+06, -2.7893e+06,
        -2.8962e+06, -3.0051e+06, -3.1160e+06, -3.2289e+06, -3.3438e+06,
        -3.4607e+06, -3.5796e+06, -3.7005e+06, -3.8234e+06, -3.9483e+06,
        -4.0752e+06, -4.2041e+06, -4.3350e+06, -4.4679e+06],
       dtype=torch.float64)
tensor(-1583840.1424, dtype=torch.f

In [6]:
#and certainly there isn't in the tensorflow, otherwise we'd be doing it twice

#what if we don'to do the log softmax?

#let's now calculate the loss
def cbpnet_multinomial_nll(logits,true_counts, len_batch=None, ignore_index=-100, mask = True, weight = .5, pos_weight = .04):

    """A loss function based on the multinomial negative log-likelihood.
    modified by me to include things like doing thE log softmax

    This loss function takes in a tensor of normalized log probabilities such
    that the sum of each row is equal to 1 (e.g. from a log softmax) and
    an equal sized tensor of true counts and returns the probability of
    observing the true counts given the predicted probabilities under a
    multinomial distribution. Can accept tensors with 2 or more dimensions
    and averages over all except for the last axis, which is the number
    of categories.

    Adapted from Alex Tseng.

    Parameters
    ----------
    logps: torch.tensor, shape=(n, ..., L)
        A tensor with `n` examples and `L` possible categories. 

    true_counts: torch.tensor, shape=(n, ..., L)
        A tensor with `n` examples and `L` possible categories.

    Returns
    -------
    loss: float
        The multinomial log likelihood loss of the true counts given the
        predicted probabilities, averaged over all examples and all other
        dimensions.
    """
    if isinstance(logits, tuple): #for wandb tracking, it inputs logits here, since then it's a tuple
        logits = logits[0]
    logps = torch.log_softmax(logits, dim=-1)
    logps = torch.log_softmax(logps, dim=-1)
    log_fact_sum = torch.lgamma(torch.sum(true_counts, dim=-1) + 1)
    log_prod_fact = torch.sum(torch.lgamma(true_counts + 1), dim=-1)
    log_prod_exp = torch.sum(true_counts * logps, dim=-1)
    return -log_fact_sum + log_prod_fact - log_prod_exp

fake_logits = np.array(range(64*1000)).reshape(64,1000) / 1000
fake_logits = torch.tensor(fake_logits)
fake_logits = torch.log_softmax(fake_logits, dim=-1)
fake_labels = np.array(range(64*1000)).reshape(64,1000) / 1000
fake_labels = torch.tensor(fake_labels)

loss = cbpnet_multinomial_nll(fake_logits, fake_labels)
print(loss)
print(loss.mean())
#wait this tells us that tf might be doing log softmax in the loss, it isn't in the model but they could still do it, this would make a lot of sense!
#this is because applying log softmax after it's already been done once leaves it unchanged!!

#so let's see if in tensorflow they apply log softmax

tensor([ 698.4480, 1170.9492, 1439.0846, 1635.6626, 1795.2100, 1932.0447,
        2053.5070, 2163.8679, 2265.8329, 2361.2251, 2451.3312, 2537.0936,
        2619.2231, 2698.2691, 2774.6652, 2848.7593, 2920.8352, 2991.1271,
        3059.8307, 3127.1112, 3193.1092, 3257.9456, 3321.7249, 3384.5379,
        3446.4645, 3507.5749, 3567.9314, 3627.5892, 3686.5979, 3745.0019,
        3802.8411, 3860.1517, 3916.9666, 3973.3155, 4029.2257, 4084.7220,
        4139.8273, 4194.5626, 4248.9472, 4302.9991, 4356.7347, 4410.1694,
        4463.3175, 4516.1922, 4568.8058, 4621.1699, 4673.2953, 4725.1921,
        4776.8697, 4828.3372, 4879.6027, 4930.6743, 4981.5594, 5032.2648,
        5082.7974, 5133.1633, 5183.3684, 5233.4184, 5283.3187, 5333.0741,
        5382.6897, 5432.1699, 5481.5190, 5530.7413], dtype=torch.float64)
tensor(3744.9462, dtype=torch.float64)


In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
def multinomial_nll(true_counts, logits):
    """Compute the multinomial negative log-likelihood
    Args:
      true_counts: observed count values
      logits: predicted logit values
    """
    counts_per_example = tf.reduce_sum(true_counts, axis=-1)
    dist = tfp.distributions.Multinomial(total_count=counts_per_example,
                                         logits=logits)
    return (-tf.reduce_sum(dist.log_prob(true_counts)) / 
            tf.cast(tf.shape(true_counts)[0], dtype=tf.float64))

fake_logits = np.array(range(64*1000)).reshape(64,1000) / 1000
#do log softmax of fake_logits
fake_logits = tf.nn.log_softmax(fake_logits, axis=-1)
fake_labels = np.array(range(64*1000)).reshape(64,1000) / 1000
#now make fake_labels a tf tensor
fake_labels = tf.convert_to_tensor(fake_labels)

loss = multinomial_nll(fake_labels, fake_logits)

2024-06-17 15:07:36.617668: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-17 15:07:41.997529: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7410 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:02:00.0, compute capability: 6.1
2024-06-17 15:07:41.999094: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 7410 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:03:00.0, compute capability: 6.1
2024-06-17 15:07:42.000409: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/r

In [2]:
loss

<tf.Tensor: shape=(), dtype=float64, numpy=3744.9462380055356>

In [ ]:
# fake_logits.shape #yeah this looks right!

loss = multinomial_nll(fake_labels, fake_logits) #this immediately crashes the kernel...
#i think it's accessing tensorflow that's the issue where it tries to accessa gpu but can't...
#also if we look at the model, we see that 

'''
This is the output that I get

<tf.Tensor: shape=(), dtype=float64, numpy=3744.9462380055356>

Exactly the same as jacob schreiber's so we know we are doing the loss correctly!!!!!
'''

: 

: 

: 

In [ ]:
#what happens if we don't apply log softmax

import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
def multinomial_nll(true_counts, logits):
    """Compute the multinomial negative log-likelihood
    Args:
      true_counts: observed count values
      logits: predicted logit values
    """
    counts_per_example = tf.reduce_sum(true_counts, axis=-1)
    dist = tfp.distributions.Multinomial(total_count=counts_per_example,
                                         logits=logits)
    return (-tf.reduce_sum(dist.log_prob(true_counts)) / 
            tf.cast(tf.shape(true_counts)[0], dtype=tf.float64))

fake_logits = np.array(range(64*1000)).reshape(64,1000) / 1000
#do log softmax of fake_logits
# fake_logits = tf.nn.log_softmax(fake_logits, axis=-1)
fake_labels = np.array(range(64*1000)).reshape(64,1000) / 1000
#now make fake_labels a tf tensor
# fake_labels = tf.convert_to_tensor(fake_labels)

loss = multinomial_nll(fake_labels, fake_logits)
print(loss)

#when we run it we get
#tf.Tensor(3744.9462380055356, shape=(), dtype=float64)...
#oh...

In [7]:
fake_logits = np.array(range(64*1000)).reshape(64,1000) / 1000
logmaxed = torch.log_softmax(torch.tensor(fake_logits), dim=-1)
print(fake_logits)
print(logmaxed)

[[0.0000e+00 1.0000e-03 2.0000e-03 ... 9.9700e-01 9.9800e-01 9.9900e-01]
 [1.0000e+00 1.0010e+00 1.0020e+00 ... 1.9970e+00 1.9980e+00 1.9990e+00]
 [2.0000e+00 2.0010e+00 2.0020e+00 ... 2.9970e+00 2.9980e+00 2.9990e+00]
 ...
 [6.1000e+01 6.1001e+01 6.1002e+01 ... 6.1997e+01 6.1998e+01 6.1999e+01]
 [6.2000e+01 6.2001e+01 6.2002e+01 ... 6.2997e+01 6.2998e+01 6.2999e+01]
 [6.3000e+01 6.3001e+01 6.3002e+01 ... 6.3997e+01 6.3998e+01 6.3999e+01]]
tensor([[-7.4486, -7.4476, -7.4466,  ..., -6.4516, -6.4506, -6.4496],
        [-7.4486, -7.4476, -7.4466,  ..., -6.4516, -6.4506, -6.4496],
        [-7.4486, -7.4476, -7.4466,  ..., -6.4516, -6.4506, -6.4496],
        ...,
        [-7.4486, -7.4476, -7.4466,  ..., -6.4516, -6.4506, -6.4496],
        [-7.4486, -7.4476, -7.4466,  ..., -6.4516, -6.4506, -6.4496],
        [-7.4486, -7.4476, -7.4466,  ..., -6.4516, -6.4506, -6.4496]],
       dtype=torch.float64)


In [9]:
#now do it again
logmaxed2 = torch.log_softmax(logmaxed, dim=-1)
print(logmaxed2)
print(np.allclose(logmaxed, logmaxed2)) #this is true, so we can do log softmax twice and it's the same as doing it once

tensor([[-7.4486, -7.4476, -7.4466,  ..., -6.4516, -6.4506, -6.4496],
        [-7.4486, -7.4476, -7.4466,  ..., -6.4516, -6.4506, -6.4496],
        [-7.4486, -7.4476, -7.4466,  ..., -6.4516, -6.4506, -6.4496],
        ...,
        [-7.4486, -7.4476, -7.4466,  ..., -6.4516, -6.4506, -6.4496],
        [-7.4486, -7.4476, -7.4466,  ..., -6.4516, -6.4506, -6.4496],
        [-7.4486, -7.4476, -7.4466,  ..., -6.4516, -6.4506, -6.4496]],
       dtype=torch.float64)
True


# explanation

In [17]:
#Logo softmax makes it such that the sum of the exponentials of any row are 1
print(torch.exp(logmaxed[0]).shape)
print(torch.sum(torch.exp(logmaxed[0]))) #this is 1 as we expected!

torch.Size([1000])
tensor(1.0000, dtype=torch.float64)


In [ ]:
#but if the sum of exponentials is 1, then leaving it as such will keep it as 1!

#however, in the torch function, they don't do the log softmax but they use probabilities which basically do that!

#I'm not too sure how it's calculated, but we know it's correct so let's use it!